# Main class for running the molecule environment

In [1]:
import gym
from gym import wrappers, logger
from MoleculeAgent import MoleculeAgent

In [2]:
def main():
    env = gym.make("gym_molecule:molecule-v0")
    
    #Create a new Molecule Agent object. 
    agent = MoleculeAgent(env.action_space)
    
    episode_count = 5
    reward = 0
    done = False

    #The 'RL algorithm' learns for 2 episodes of undefined length.
    for i in range(0, episode_count):
        print("EPISODE:", i)
        
        #The env resets.
        ob = env.reset()
        print("Env original state: ", ob)
        
        #While what is True? Maybe While done is False?
        for x in range(10):
            
            #The agent chooses its action based on the tuple from the current env state.
            action = agent.act(ob, reward, done)
            print("Agent's chosen action: ", action) 
            
            #The new tuple is returned because the env has considered the agent's action.
            ob, reward, done, _ = env.step(action)
            print("Env new state:", ob, "Reward to agent:",reward, "Done:",done, "Info: ", _)
            
            #The env renders. 
            env.render()
            
            
        
            #If the env reaches a terminating state, the current iteration ends and the next 
            #episode begins. 
            """"if done:
                print()
                break"""
        
    env.close()
    
    
if __name__ == '__main__':
    main()

EPISODE: 0
Env original state:  0.0
Agent's chosen action:  5
5
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  0
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  5
5
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  2
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  3
3
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  3
3
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  2
calculating Reward
Env new state: 20.0 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  2
calculating Reward
Env new state: 14.2857 Reward to agent: 1 Done: False Info:  {}
Agent's chosen action:  4
4
calculating Reward
Env new state: 14.2857 Reward to agent: 1 Done: False Info:  {}


RDKit ERROR: [10:05:02] Explicit valence for atom # 1 F, 2, is greater than permitted
RDKit ERROR: [10:05:02] Explicit valence for atom # 4 Cl, 2, is greater than permitted
RDKit ERROR: [10:05:02] Explicit valence for atom # 5 Cl, 2, is greater than permitted
RDKit ERROR: [10:05:02] Explicit valence for atom # 8 Cl, 2, is greater than permitted
